In [27]:
# import packages
import pandas as pd
import re
import string
import pickle

In [28]:
# read csv file for car data

# download the data from https://data.world/data-society/used-cars-data
df = pd.read_csv('/home/anuraaga/Documents/Projects/pricePredictor/dataset/dataset_car/autos.csv', encoding_errors='ignore')

- dateCrawled : when this ad was first crawled, all field-values are taken from this date
- name : "name" of the car
- seller : private or dealer
- offerType
- price : the price on the ad to sell the car
- abtest
- vehicleType
- yearOfRegistration : at which year the car was first registered
- gearbox
- powerPS : power of the car in PS
- model
- kilometer : how many kilometers the car has driven
- monthOfRegistration : at which month the car was first registered
- fuelType
- brand
- notRepairedDamage : if the car has a damage which is not repaired yet
- dateCreated : the date for which the ad at ebay was created
- nrOfPictures : number of pictures in the ad
- postalCode
- lastSeenOnline : when the crawler saw this ad last online

In [29]:
# information of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          371528 non-null  object
 1   name                 371528 non-null  object
 2   seller               371528 non-null  object
 3   offerType            371528 non-null  object
 4   price                371528 non-null  int64 
 5   abtest               371528 non-null  object
 6   vehicleType          333659 non-null  object
 7   yearOfRegistration   371528 non-null  int64 
 8   gearbox              351319 non-null  object
 9   powerPS              371528 non-null  int64 
 10  model                351044 non-null  object
 11  kilometer            371528 non-null  int64 
 12  monthOfRegistration  371528 non-null  int64 
 13  fuelType             338142 non-null  object
 14  brand                371528 non-null  object
 15  notRepairedDamage    299468 non-nu

In [30]:
df.isnull().sum()

dateCrawled                0
name                       0
seller                     0
offerType                  0
price                      0
abtest                     0
vehicleType            37869
yearOfRegistration         0
gearbox                20209
powerPS                    0
model                  20484
kilometer                  0
monthOfRegistration        0
fuelType               33386
brand                      0
notRepairedDamage      72060
dateCreated                0
nrOfPictures               0
postalCode                 0
lastSeen                   0
dtype: int64

In [32]:
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [31]:
df[['name', 'brand', 'model']]

,name,brand,model
0,Golf_3_1.6,volkswagen,golf
1,A5_Sportback_2.7_Tdi,audi,NaN
2,"Jeep_Grand_Cherokee_""Overland""",jeep,grand
3,GOLF_4_1_4__3TRER,volkswagen,golf
4,Skoda_Fabia_1.4_TDI_PD_Classic,skoda,fabia
...,...,...,...
371523,Suche_t4___vito_ab_6_sitze,sonstige_autos,NaN
371524,Smart_smart_leistungssteigerung_100ps,smart,fortwo
371525,Volkswagen_Multivan_T4_TDI_7DC_UY2,volkswagen,transporter
371526,VW_Golf_Kombi_1_9l_TDI,volkswagen,golf


In [33]:
def clean_name(name):
    modelname = name.replace('_', ' ')
    # modelname = modelname.lower()
    modelname = modelname.strip()
    return modelname

df['name'] = df['name'].apply(clean_name)

In [34]:
df

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf 3 1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5 Sportback 2.7 Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep Grand Cherokee ""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF 4 1 4 3TRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda Fabia 1.4 TDI PD Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche t4 vito ab 6 sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart smart leistungssteigerung 100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen Multivan T4 TDI 7DC UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW Golf Kombi 1 9l TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


In [35]:
# find if any character in object dtype column contains special characters
cols = df.columns[df.dtypes == object]
cols_with_punctuations = []
for col in cols:
    any_punctuation = any(char in string.punctuation for text in df[col].dropna() for char in text)
    if any_punctuation:
        cols_with_punctuations.append(col)

cols_with_punctuations

['dateCrawled', 'name', 'model', 'brand', 'dateCreated', 'lastSeen']

In [36]:
# substitute those characters with blank character
for col in cols_with_punctuations:
    df[col] = df[col].apply(lambda text: re.sub(f'[{re.escape(string.punctuation)}]', '', str(text)) if pd.notnull(text) else text)

In [37]:
df['name'] = df['name'].str.strip()

In [55]:
cldf = df[df['name']!='']

In [56]:
cldf

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,20160324 115217,Golf 3 16,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,20160324 000000,0,70435,20160407 031657
1,20160324 105845,A5 Sportback 27 Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,20160324 000000,0,66954,20160407 014650
2,20160314 125221,Jeep Grand Cherokee Overland,privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,20160314 000000,0,90480,20160405 124746
3,20160317 165404,GOLF 4 1 4 3TRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,20160317 000000,0,91074,20160317 174017
4,20160331 172520,Skoda Fabia 14 TDI PD Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,20160331 000000,0,60437,20160406 101721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,20160314 174827,Suche t4 vito ab 6 sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstigeautos,NaN,20160314 000000,0,39576,20160406 004652
371524,20160305 195621,Smart smart leistungssteigerung 100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,20160305 000000,0,26135,20160311 181712
371525,20160319 185712,Volkswagen Multivan T4 TDI 7DC UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,20160319 000000,0,87439,20160407 071526
371526,20160320 194108,VW Golf Kombi 1 9l TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,20160320 000000,0,40764,20160324 124521


In [57]:
cldf2 = cldf.copy()

In [58]:
cldf2['brand']=cldf2['brand'].apply(lambda x: x.capitalize())

In [59]:
cldf2.at[1,'name'].split()[0] != cldf2.at[371495, 'brand']

True

In [60]:
nan_model_rows = cldf2[cldf2['model'].isna()]
nan_model_rows

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
1,20160324 105845,A5 Sportback 27 Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,Audi,ja,20160324 000000,0,66954,20160407 014650
60,20160329 154815,TAUSCHE BMW E38 740i g SUV GELNDEWAGEN LESEN ...,privat,Angebot,1,control,suv,1994,manuell,286,NaN,150000,11,NaN,Sonstigeautos,NaN,20160329 000000,0,53721,20160406 014438
83,20160403 125645,Bastler Fahrzeug mit Rest Tuev,privat,Angebot,350,control,kleinwagen,1997,manuell,54,NaN,150000,3,NaN,Fiat,ja,20160403 000000,0,45665,20160405 114713
119,20160320 185327,Polo 6n Karosse zu verschenken,privat,Angebot,0,test,kleinwagen,1999,NaN,0,NaN,5000,0,benzin,Volkswagen,NaN,20160320 000000,0,37520,20160407 024522
139,20160327 205123,Audi 80 5 Zylinder b4 liebhaberfahrzeug,privat,Angebot,1450,control,limousine,1992,manuell,136,NaN,150000,0,NaN,Audi,nein,20160327 000000,0,38709,20160405 201731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371399,20160307 163742,Fiat Stilo Abarth 24 5zylinder Seelespeed,privat,Angebot,560,control,kleinwagen,2001,automatik,170,NaN,90000,0,benzin,Fiat,ja,20160307 000000,0,55743,20160312 214525
371476,20160315 135234,ALPINA D3 Touring,privat,Angebot,9400,control,kombi,2007,manuell,200,NaN,150000,4,diesel,Sonstigeautos,ja,20160315 000000,0,91186,20160317 081625
371495,20160331 195233,Opel Tigra,privat,Angebot,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,Opel,NaN,20160331 000000,0,41470,20160406 141804
371509,20160311 234032,LT 35 DIESEL Gruene Plaketteletzte Gelegenheit,privat,Angebot,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,Volkswagen,nein,20160311 000000,0,87700,20160312 141651


In [61]:
for index, row in nan_model_rows.iterrows():
    if cldf2.at[index,'name'].split()[0] != cldf2.at[index, 'brand']:
        cldf2.at[index,'name'] = row['brand'] + " " + row['name']

In [62]:
cldf2

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,20160324 115217,Golf 3 16,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,Volkswagen,NaN,20160324 000000,0,70435,20160407 031657
1,20160324 105845,Audi A5 Sportback 27 Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,Audi,ja,20160324 000000,0,66954,20160407 014650
2,20160314 125221,Jeep Grand Cherokee Overland,privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,Jeep,NaN,20160314 000000,0,90480,20160405 124746
3,20160317 165404,GOLF 4 1 4 3TRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,Volkswagen,nein,20160317 000000,0,91074,20160317 174017
4,20160331 172520,Skoda Fabia 14 TDI PD Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,Skoda,nein,20160331 000000,0,60437,20160406 101721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,20160314 174827,Sonstigeautos Suche t4 vito ab 6 sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,Sonstigeautos,NaN,20160314 000000,0,39576,20160406 004652
371524,20160305 195621,Smart smart leistungssteigerung 100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,Smart,nein,20160305 000000,0,26135,20160311 181712
371525,20160319 185712,Volkswagen Multivan T4 TDI 7DC UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,Volkswagen,nein,20160319 000000,0,87439,20160407 071526
371526,20160320 194108,VW Golf Kombi 1 9l TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,Volkswagen,NaN,20160320 000000,0,40764,20160324 124521


In [66]:
cldf2.isnull().sum()

dateCrawled                0
name                       0
seller                     0
offerType                  0
price                      0
abtest                     0
vehicleType            37867
yearOfRegistration         0
gearbox                20207
powerPS                    0
model                  20483
kilometer                  0
monthOfRegistration        0
fuelType               33383
brand                      0
notRepairedDamage      72058
dateCreated                0
nrOfPictures               0
postalCode                 0
lastSeen                   0
dtype: int64

In [65]:
cldf2.at[306825, 'name']

'Lancia Lybra 18 LX'

In [67]:
newdata = cldf2.drop(['dateCrawled', 'dateCreated', 'lastSeen', 'postalCode', 'nrOfPictures', 'model', 'powerPS', 'offerType'], axis=1)


In [68]:
newdata.isnull().sum()

name                       0
seller                     0
price                      0
abtest                     0
vehicleType            37867
yearOfRegistration         0
gearbox                20207
kilometer                  0
monthOfRegistration        0
fuelType               33383
brand                      0
notRepairedDamage      72058
dtype: int64

In [24]:
pickle.dump(newdata, open('/home/anuraaga/Documents/Projects/pricePredictor/dataset/dataset_car/autos_v1.pkl', 'wb'))